# 使用 CAMEL 框架搭建 Agent (Kimi k2.5 实践)

本教程将带你使用 [CAMEL-AI](https://www.camel-ai.org/) 框架，并结合 Moonshot AI 的 **Kimi k2.5** 模型来构建你的第一个 AI Agent。

## 1. 环境准备

首先，我们需要安装 `camel-ai` 库。

In [ ]:
# 在某些环境下，可能需要使用 python3 -m pip
!python3 -m pip install camel-ai python-dotenv

## 2. 配置 Kimi k2.5 API

Kimi k2.5 的 API 与 OpenAI 兼容。你需要准备好 Moonshot AI 的 API Key。

你可以从 [Moonshot AI 开放平台](https://platform.moonshot.ai/) 获取。

In [2]:
import os
from getpass import getpass

if "MOONSHOT_API_KEY" not in os.environ:
    os.environ["MOONSHOT_API_KEY"] = getpass("请输入你的 Moonshot API Key: ")

# 重要：CAMEL 有时会强制检查 OPENAI_API_KEY 环境变量
os.environ["OPENAI_API_KEY"] = os.environ["MOONSHOT_API_KEY"]

KIMI_BASE_URL = "https://api.moonshot.ai/v1"
KIMI_MODEL_TYPE = "kimi-k2.5"

## 3. 初始化 Kimi k2.5 模型

使用 CAMEL 的 `ModelFactory` 来创建一个 OpenAI 兼容的模型实例。

In [3]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

kimi_model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=KIMI_MODEL_TYPE,
    model_config_dict={
        "api_key": os.environ["MOONSHOT_API_KEY"],
        "base_url": KIMI_BASE_URL,
    },
)

print(f"成功初始化模型: {KIMI_MODEL_TYPE}")

成功初始化模型: kimi-k2.5


## 4. 构建基础 ChatAgent

我们先创建一个简单的对话 Agent。

In [4]:
from camel.agents import ChatAgent
from camel.messages import BaseMessage

system_message = BaseMessage.make_assistant_message(
    role_name="Assistant",
    content="你是一个由 Kimi k2.5 驱动的智能助手，能够提供准确、专业的建议。",
)

agent = ChatAgent(
    system_message=system_message,
    model=kimi_model,
)

user_msg = BaseMessage.make_user_message(role_name="User", content="请简要介绍一下 CAMEL-AI 框架的主要特点。")
response = agent.step(user_msg)

print(response.msgs[0].content)

2026-02-08 17:00:36,456 - root - WARNING - Unknown model 'kimi-k2.5': context window size not defined. Defaulting to 999_999_999.
2026-02-08 17:00:39,631 - camel.models.model_manager - ERROR - Error processing with model: <camel.models.openai_model.OpenAIModel object at 0x105ed22d0>
2026-02-08 17:00:39,631 - camel.camel.agents.chat_agent - ERROR - Model error: kimi-k2.5


TypeError: Completions.create() got an unexpected keyword argument 'api_key'

## 5. 进阶：角色扮演 (Role-Playing)

CAMEL 框架最核心的特点是多智能体协同（Role-Playing）。

In [ ]:
from camel.societies import RolePlaying

task_prompt = "开发一个简单的待办事项(Todo List)命令行应用，使用 Python 编写。"

role_play_session = RolePlaying(
    assistant_role_name="Python 程序员",
    user_role_name="项目经理",
    task_prompt=task_prompt,
    model_platform=ModelPlatformType.OPENAI,
    model_type=KIMI_MODEL_TYPE,
    model_config_dict={
        "api_key": os.environ["MOONSHOT_API_KEY"],
        "base_url": KIMI_BASE_URL,
    },
)

print(f"任务: {task_prompt}")

chat_turn_limit = 3
input_msg = role_play_session.init_chat()

for i in range(chat_turn_limit):
    assistant_response, user_response = role_play_session.step(input_msg)
    
    print(f"\n第 {i+1} 轮 ---")
    print(f"项目经理: {user_response.msg.content[:200]}...")
    print(f"程序员: {assistant_response.msg.content[:200]}...")
    
    input_msg = assistant_response.msg

## 6. 总结

通过这个 Notebook，你已经学会了：
1. 如何配置 CAMEL-AI 使用自定义的 OpenAI 兼容模型（Kimi k2.5）。
2. 如何创建单个 ChatAgent 进行对话。
3. 如何使用 `RolePlaying` 启动多智能体协作。